# Simple Solution : Assymetric VanLaar Simple solution:  SymPy Code Generation

This notebook generates a model for a 2 component solid solution using an Assymetric VanLaar model for the excess

In [ ]:
import os,sys
import pandas as pd
import numpy as np
import sympy as sym
import hashlib
import time
sym.init_printing()

Required ENKI packages

In [ ]:
from thermocodegen.coder import coder

### let's set up some directory names for clarity

In [ ]:
HOME_DIR = os.path.abspath(os.curdir)
SPUD_DIR = HOME_DIR+'/../phases'

try:
    os.mkdir(SPUD_DIR)
except:
    pass

Set a reference string for this Notebook

In [ ]:
reference = 'Thermocodegen-v0.6/share/thermocodegen/examples/Systems/MgFeSiO4_stixrued/notebooks/Generate_phases.ipynb'

## Ideal Solution properties
There are Three terms:
- Terms describing standard state contributions
- Terms describing the configurational entropy of solution
- Terms describing the Excess Entropy using a van laar model

Assumptions:
- There are $c$ components in the system
- There are as many endmember species, $s$, as there are components
- The configurational entropy is described as a simple $x_i log(x_i)$ sum
- The excess entropy is discribed by a van Laar model used in Stixrude (ref) 199

### Number of solution components
This notebook illustrates a two component solution

In [ ]:
c = 2

### Create a simple solution model
... with the specified number of endmember thermodynamic components

In [ ]:
model = coder.SimpleSolnModel.from_type(nc=c)

### Retrieve primary compositional variables
- $n$ is a vector of mole numbers of each component  
- $n_T$ is the total number of moles in the solution

### and construct a derived mole fraction variable
- $X$ is a vector of mole fractions of components in the system

In [ ]:
n = model.n
nT = model.nT
X = n/nT
n, nT, X

### Retrieve the temperature, pressure, and standard state chemical potentials
- $T$ is temperature in $K$
- $P$ is pressure in $bars$
- $\mu$ in Joules

In [ ]:
T = model.get_symbol_for_t()
P = model.get_symbol_for_p()
mu = model.mu
T,P,mu

Check model dictionary

In [ ]:
model.model_dict

### Define the standard state contribution to solution properties

In [ ]:
G_ss = (n.transpose()*mu)[0]
G_ss

### Define configurational entropy and configurational Gibbs free energy

Mixing on sites.  The Stixrude model for the configurational Gibbs Free Energy can be written

$$
    G_{conf} = RT \mathbf{m}^T\text{diag}(\mathbf{x}^*)\log(\mathbf{x}^*)
$$

where $\mathbf{m}$ is a vector containing the site multiplicity of length `ns`

For the MgFeSi$_2$O$_4$ system there are 2 sites each with multiplicity 2 so

$$
    \mathbf{m}^T=\begin{bmatrix} 2 & 2\\\end{bmatrix}
$$

$$
\mathbf{x}^* = C\mathbf{x}
$$ 

is the vector of mole fractions on each site and $C$ is a matrix of site Coefficients.

Again for this system $C=I$

In [ ]:
ns = 2
G_config, R = sym.symbols('S_config R')
C = sym.eye(ns)
X_o = C*X
X_o

In [ ]:
M = sym.Matrix([2,2])
G = nT*R*T*M.T*sym.diag(*X_o)*X_o.applyfunc(sym.log)
G_config = G[0].simplify()
G_config

### Define Excess Gibbs free energy of mixing

Stixrude uses a van Laar model for describing the excess Gibbs free energy

$$
    G_{ex} = \mathbf{x}^T\mathbf{d}
    \left[ \mathbf{\Phi}^T W^*\mathbf{\Phi}\right]
$$

where $\mathbf{d}$ is a vector containing species weights (length $c$, number of components)

$$
    \mathbf{\Phi} = \frac{\text{diag}(\mathbf{d})\mathbf{x}}{\mathbf{d}^T\mathbf{x}}
$$

is a vector of $d$ weighted mol fractions and

$$
    W^*_{i,j} = \frac{W_{i,j}}{d_i + d_j}
$$

is a $\mathbf{d}$ weighted Margules matrix $W$ of species interaction terms.

For the MgFeSi$_2$O$_4$ system $W$ is a $2\times2$ symmetric matrix with only one off-diagonal term $W_{1,2}=W_{2,1}$ and $\mathbf{d}^T = \begin{bmatrix} 1 & 1\\ \end{bmatrix}$


In [ ]:
W12 = sym.Symbol('W_12')
d = sym.Matrix([1,1])
W = sym.Matrix([[0, W12], [W12, 0]])
for i in range(2):
    for j in range(2):
        W[i,j] = W[i,j]/(d[i] + d[j])
Phi = sym.diag(*d)*X/(X.T*d)
Phi = Phi.simplify()
G_excess = nT*X.T*d*Phi.T*W*Phi
G_excess = G_excess.simplify()[0]
G_excess

In [ ]:
params=['R','W_12']
units = ['J/K/mol', 'J/mol']
symparam =[R, W12]

In [ ]:
print(list(zip(params,units,symparam)))

### Define the Gibbs free energy of solution

In [ ]:
G = G_ss + G_config + G_excess
G

### Add the Gibbs free energy of solution to the model

In [ ]:
model.add_potential_to_model('G',G,list(zip(params,units,symparam)))

### let's inspect the dictionary and unset parameters

In [ ]:
model.model_dict

In [ ]:
values_dict = model.get_values()
values_dict

## Define Parameters of a Solution models for various phases of MgFeSiO4 solid solutions
Components
1. Mg endmembers, ${\rm{Mg_2SiO_4}}$
2. Fe endmembers, ${\rm{Fe_2SiO_4}}$

* assign a formula string for code generation  
    * here the solution formula is Mg$_x$Fe$_{2-x}$SiO$_4$ where $x\in[0,2]$
* assign a conversion string to map element concentrations to moles of end members
    * here endmember 0 is Forsterite, ${\rm{Mg_2SiO_4}}$ so 1 mole of Forsterite will be 0.5* the number of Mg atoms
    * likewise for endmember 1, Fayalite,  1 mole of Fa will be 0.5* the number of Fe atoms whic gives the conversion_string for moles of endmember 0, and 1 as 
    
    `conversion_string=['[0]=0.5*[Mg]', '[1]=0.5*[Fe]']`


In [ ]:
values_dict.update(dict(formula_string='Mg[Mg]Fe[Fe]SiO4',
                        conversion_string=['[0]=0.5*[Mg]', '[1]=0.5*[Fe]'],
                        test_string = ['[0] > 0.0', '[1] > 0.0'],
                        R=8.31446261815324, T_r=300.))
values_dict

### Add additional parameters for solid phases Olivine, Wadsleyite and Ringwoodite

In [ ]:
Ol_dict = dict(name='Olivine',abbrev='Ol',
                        reference=reference,
                        endmembers = ['Forsterite_stixrude', 'Fayalite_stixrude'],
                        W_12 = 7600.)
Wa_dict = dict(name='Wadsleyite',abbrev='Wa',
                        reference=reference,
                        endmembers = ['MgWadsleyite_stixrude', 'FeWadsleyite_stixrude'],
                        W_12 = 16500. )
Ri_dict = dict(name='Ringwoodite',abbrev='Ri',
                        reference=reference,
                        endmembers = ['MgRingwoodite_stixrude', 'FeRingwoodite_stixrude'],
                        W_12 = 9100.)

## Generate Spud XML files

### dump spudfiles

In [ ]:
for d in [ Ol_dict, Wa_dict, Ri_dict]:
    values_dict.update(d)
    model.set_values(values_dict)
    file = model.to_xml(path=SPUD_DIR)
    print(file)